In [1]:
import pandas as pd
import os
import sys
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
# Append the project root path to sys.path
sys.path.append(os.path.abspath(".."))

In [2]:
# --- Task 2: Model Building and Training ---

# 1. Load Merged Data (from Task 1)
file_path = r'C:\Users\user\Desktop\Kifiya\Adey-Innovations-Fraud-Mgt\data\preprocessed_data.csv'
if not os.path.exists(file_path):
    print("Error: preprocessed_data.csv not found. Please run Task 1 first.")
    exit()

merged_data = pd.read_csv(file_path)

# Columns to remove (replace with your actual column names)
columns_to_remove = ['signup_time', 'purchase_time', 'device_id', 'country']

# 2. Feature and Target Separation (Fraud Data)
X_fraud = merged_data.drop('class', axis=1)
y_fraud = merged_data['class'].astype(int)  # Ensure y is numeric (0 or 1)
X_fraud = X_fraud.drop(columns=columns_to_remove, errors='ignore')

# 3. Train-Test Split (Fraud Data)
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

# 4. Scaling (Fraud Data)
scaler = StandardScaler()
numeric_features = ["purchase_value", "age", "ip_address", "purchase_frequency_proxy"]
merged_data[numeric_features] = scaler.fit_transform(merged_data[numeric_features])
X_fraud_train = scaler.fit_transform(X_fraud_train)
X_fraud_test = scaler.transform(X_fraud_test)

# 5. Model Selection and Training (with MLflow - Fraud Data)
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced'),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

for name, model in models.items():
    with mlflow.start_run(run_name=f"{name} (Fraud Data)"):
        model.fit(X_fraud_train, y_fraud_train)
        y_pred = model.predict(X_fraud_test)

        accuracy = accuracy_score(y_fraud_test, y_pred)
        report = classification_report(y_fraud_test, y_pred)

        mlflow.log_param("model", name)
        mlflow.log_metric("accuracy", accuracy)

        # Save the classification report as a text file and log it
        with open("classification_report.txt", "w") as f:
            f.write(report)
        mlflow.log_artifact("classification_report.txt")

        mlflow.sklearn.log_model(model, "model")

        print(f"{name} (Fraud Data) Results:")
        print(f"Accuracy: {accuracy}")
        print("Classification Report:\n", report)
        print("-" * 50)

2025/02/14 11:58:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression (Fraud Data) Results:
Accuracy: 0.5343943354398968
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.54      0.68     27393
           1       0.10      0.52      0.17      2830

    accuracy                           0.53     30223
   macro avg       0.51      0.53      0.42     30223
weighted avg       0.84      0.53      0.63     30223

--------------------------------------------------


2025/02/14 11:58:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree (Fraud Data) Results:
Accuracy: 0.9012672467987957
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.95     27393
           1       0.48      0.56      0.51      2830

    accuracy                           0.90     30223
   macro avg       0.71      0.75      0.73     30223
weighted avg       0.91      0.90      0.90     30223

--------------------------------------------------


2025/02/14 11:59:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest (Fraud Data) Results:
Accuracy: 0.95486880852331
Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.98     27393
           1       0.99      0.52      0.68      2830

    accuracy                           0.95     30223
   macro avg       0.97      0.76      0.83     30223
weighted avg       0.96      0.95      0.95     30223

--------------------------------------------------


2025/02/14 11:59:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting (Fraud Data) Results:
Accuracy: 0.9069251894252721
Classification Report:
               precision    recall  f1-score   support

           0       0.91      1.00      0.95     27393
           1       0.95      0.01      0.01      2830

    accuracy                           0.91     30223
   macro avg       0.93      0.50      0.48     30223
weighted avg       0.91      0.91      0.86     30223

--------------------------------------------------


2025/02/14 12:00:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLP (Fraud Data) Results:
Accuracy: 0.9497402640373226
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97     27393
           1       0.89      0.53      0.66      2830

    accuracy                           0.95     30223
   macro avg       0.92      0.76      0.82     30223
weighted avg       0.95      0.95      0.94     30223

--------------------------------------------------


In [4]:
X_fraud_train

array([[-1.4161757 ,  1.18657302, -0.59946771, ..., -0.56797302,
        -0.15815477, -0.44022763],
       [ 0.33149434, -0.84276313, -1.29584246, ..., -0.56797302,
        -0.15815477,  2.27155212],
       [-1.36156101, -0.84276313, -0.13521787, ...,  1.76064701,
        -0.15815477, -0.44022763],
       ...,
       [ 0.82302654, -0.84276313,  0.90934427, ...,  1.76064701,
        -0.15815477, -0.44022763],
       [ 1.15071467, -0.84276313,  0.44509443, ...,  1.76064701,
        -0.15815477, -0.44022763],
       [ 0.33149434,  1.18657302, -0.71553016, ..., -0.56797302,
        -0.15815477, -0.44022763]])

In [15]:
# --- Credit Card Data Processing ---

# 1. Load Data (Credit Card)
try:
    credit_data = pd.read_csv(r'C:\Users\user\Desktop\Kifiya\Adey-Innovations-Fraud-Mgt\data\creditcard.csv')
except FileNotFoundError:
    print("Error: creditcard.csv not found.")
    exit()

# 2. Feature and Target Separation (Credit Card)
X_credit = credit_data.drop('Class', axis=1)
y_credit = credit_data['Class'].astype(int) # Ensure y is numeric (0 or 1)

# 3. Train-Test Split (Credit Card)
X_credit_train, X_credit_test, y_credit_train, y_credit_test = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit
)

# 4. Scaling (Credit Card)
scaler_credit = StandardScaler()
X_credit_train = scaler_credit.fit_transform(X_credit_train)
X_credit_test = scaler_credit.transform(X_credit_test)

# 5. Model Selection and Training (with MLflow - Credit Card)
models_cred = {
    "Logistic Regression": LogisticRegression(class_weight='balanced'),  # Increase max_iter if needed
    "Decision Tree": DecisionTreeClassifier(),
}

for name, model in models_cred.items():  # Re-using the same models dictionary
    with mlflow.start_run(run_name=f"{name} (Credit Card Data)"):
        model.fit(X_credit_train, y_credit_train)
        y_pred = model.predict(X_credit_test)

        accuracy = accuracy_score(y_credit_test, y_pred)
        report = classification_report(y_credit_test, y_pred)

        mlflow.log_param("model", name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_text(report, "classification_report.txt")
        mlflow.sklearn.log_model(model, "model")

        print(f"{name} (Credit Card Data) Results:")
        print(f"Accuracy: {accuracy}")
        print("Classification Report:\n", report)
        print("-" * 50)

2025/02/11 16:00:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression (Credit Card Data) Results:
Accuracy: 0.9755275446789088
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962

--------------------------------------------------


2025/02/11 16:00:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree (Credit Card Data) Results:
Accuracy: 0.9990344440153085
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.72      0.71      0.72        98

    accuracy                           1.00     56962
   macro avg       0.86      0.86      0.86     56962
weighted avg       1.00      1.00      1.00     56962

--------------------------------------------------
